In [1]:
import os
import numpy 
import pandas as pd
import torch
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.document_loaders import WikipediaLoader
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter, RecursiveCharacterTextSplitter
import myloadlib
from myloadlib import loadDir, loadFile, loadWiki, loadYoutube, readAPI
from langchain_community.document_loaders.csv_loader import CSVLoader

# Data Ingestion

In [2]:
raw_docs = []

In [3]:
raw_docs.extend(myloadlib.loadFile("https://www.providenceschools.org/cms/lib/RI01900003/Centricity/Domain/723/Automotive%20training.pdf"))
len(raw_docs)

978

In [4]:
raw_docs.extend(myloadlib.loadFile("https://transportation.wv.gov/highways/training/TrainingDocuments/Crawfords_Auto_Repair_Guide.pdf"))
len(raw_docs)

1062

In [5]:
raw_docs.extend(myloadlib.loadFile("https://www.missskirtich.com/uploads/2/3/3/7/23374820/how_to_change_your_car_oil.pdf"))
len(raw_docs)

1066

In [6]:
raw_docs.extend(myloadlib.loadFile("https://techinfo.honda.com/rjanisis/pubs/OM/IT9393/IT9393O00108A.pdf"))
len(raw_docs)

1077

In [7]:
raw_docs.extend(CSVLoader(file_path="./data/cars.csv").load())
len(raw_docs)

38362

In [8]:
raw_docs.extend(CSVLoader(file_path="./data/products.csv").load())
len(raw_docs)

42289

In [9]:
raw_docs.extend(myloadlib.loadFile("https://di-uploads-pod5.dealerinspire.com/autocitysd/uploads/2017/04/The-Ultimate-Used-Car-Buying-Guide.pdf"))
len(raw_docs)

42366

In [10]:
raw_docs.extend(myloadlib.loadFile("https://nacpi.net/sites/default/files/2023-05/Buying%20a%20Used%20Car.pdf"))
len(raw_docs)

42386

In [11]:
raw_docs.extend(myloadlib.loadFile("https://www.icandrive.com/wp-content/uploads/2019/01/How-to-Change-a-Flat-Tire-Student-Handout.pdf"))
len(raw_docs)

42388

In [12]:
raw_docs.extend(myloadlib.loadFile("https://irp.cdn-website.com/a8e0ea67/files/uploaded/Caring_For_Your_Car_The_Easy_Way_copy.pdf"))
len(raw_docs)

42409

In [13]:
raw_docs.extend(myloadlib.loadYoutube("https://www.youtube.com/watch?v=0KDMdYww4VE", "en"))
len(raw_docs)

42410

# splitting documents into chuncks

In [14]:
splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=102)

In [15]:
split_docs = splitter.split_documents(raw_docs)

# Emedding chuncks

In [16]:
from langchain import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

In [17]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

In [18]:
embedder = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# storing in vector db

In [79]:
pip show pydantic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: pydantic
Version: 2.7.1
Summary: Data validation using Python type hints
Home-page: 
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>
License: 
Location: /Users/vincentbuchholz/opt/anaconda3/lib/python3.9/site-packages
Requires: annotated-types, pydantic-core, typing-extensions
Required-by: chromadb, chromadb-client, confection, fastapi, langchain, langchain-core, langsmith, spacy, thinc, weasel, weaviate-client
Note: you may need to restart the kernel to use updated packages.


In [19]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [20]:
weaviate_url = "http://127.0.0.1:8080"

In [21]:
weaviate_client = weaviate.connect_to_local()

In [22]:
weaviate_client.is_ready()

True

In [23]:
db = WeaviateVectorStore.from_documents(split_docs, embedder, client=weaviate_client,index_name="car_info")

# Searching

### similarity search

In [49]:
query = "My car makes wired noises, what might be the issue?"

In [50]:
docs = db.similarity_search(query)

# Print the first 100 characters of each result
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:100] + "...")



Document 1:
AUTOMOTIVE TECHNICIA N TRAINING –  © TOM DENTON  
952 to the customer. Advise customers if anything ...

Document 2:
Page | 20 
 
 7. Never tow your vehicle with the drive wh eels on the ground . Front wheel drive veh...

Document 3:
AUTOMOTIVE TECHNICIA N TRAINING –  © TOM DENTON  
647 Alternator Drive Belt  Check the alternator dr...

Document 4:
AUTOMOTIVE TECHNICIA N TRAINING –  © TOM DENTON  
262 Pinking This i s a characteristic noise 
cause...


In [55]:
query = "good Bosch car parts"

In [56]:
docs = db.similarity_search(query)

# Print the first 100 characters of each result
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:100] + "...")



Document 1:
product_name: Rear muffler
product_description: 308505700
brand_name: Bosch
product_category: Parts...

Document 2:
product_name: Coil spring
product_description: For vehicles without M-technology.OE: 33536767337
bra...

Document 3:
product_name: Headlamps
product_description: For vehicles with headlight range control (electric) if...

Document 4:
product_name: Oil filters
product_description: OE: 13281621
brand_name: Bosch
product_category: Part...


In [28]:
query = "Whats the price of a used Ford"

In [30]:
docs = db.similarity_search(query)

# Print the first 100 characters of each result
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:100] + "...")



Document 1:
Make: Ford
Model: F-150
Price: 699900 DKK
ModelYear: 2018
FuelType: Benzin
Reg: 2016
Kilometers: 144...

Document 2:
Make: Ford
Model: S-MAX
Price: 39990 DKK
ModelYear: 2007
FuelType: Benzin
Reg: 2007
Kilometers: 3110...

Document 3:
Make: Ford
Model: S-MAX
Price: 299500 DKK
ModelYear: 2019
FuelType: Diesel
Reg: 2019
Kilometers: 198...

Document 4:
Make: Ford
Model: S-MAX
Price: 249999 DKK
ModelYear: 2019
FuelType: Diesel
Reg: 2019
Kilometers: 178...


In [31]:
query = "which product could be used to clean my car"

In [32]:
docs = db.similarity_search(query)

# Print the first 100 characters of each result
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:100] + "...")



Document 1:
product_name: Super Cleaner, 500 ml
product_description: Cleaning agent that effectively removes dir...

Document 2:
AUTOMOTIVE TECHNICIA N TRAINING –  © TOM DENTON  
97 for cleaning may strip the protective 
coatings...

Document 3:
product_name: Interior cleaning, 300 ml
product_description: Clean the dashboard and plastic interio...

Document 4:
product_name: Interior cleaning
product_description: Clean the dashboard and plastic interior surfac...


## MMR 

In [33]:
retriever_w = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)

In [34]:
query = "How do i change the oil of my car?"
response = retriever_w.invoke(query)
response[0]

Document(page_content='Page | 14 \n \n better and you’ll enjoy improved gas mileage.  \n \n44. Check Your Hoses  \n \nHoses become brittle and can break with time. When the  car is shut off and has cooled, squeeze the hoses. If they  \nare extremely stiff, make a crunch sound, are soft or sticky, have bulges, or looked collapsed in any section, it \nmeans the hose is weak and should be replaced. You should never drive with a damaged coolant hose and your \nengine could overheat and you coul d land up with a very expensive repair bill.  \n \n45. Belt Tension  \n \nYou should check the tension of all your belts. You should also check for wear. You’ll find belts that run your AC \ncompressor, power steering pump, and water pump. To check for tension press in the center of the belt where the \nlongest exposed part is found. If you can depress the belt ½” to 1” or 13mm to 25mm, but no more, the tension is \ngood. If not you can either take your car to auto shop for adjustment or if you are 

In [57]:
query = "My car makes wired noises, what might be the issue?"
response = retriever_w.invoke(query)
response[0]

Document(page_content='AUTOMOTIVE TECHNICIA N TRAINING –  © TOM DENTON  \n952 to the customer. Advise customers if anything will need \nattention before the next scheduled service interval.  \nVehicle Condition Respect your custo mer’s vehicle and take \nprecautions to keep it clean. Repairing or checking some \nsystems, is likely to involve you working under the vehicle, and \nthen sitting in the driver’s seat. Use seat covers and ensure the \nsteering wheel is clean when you have finished.  \nDescribing Noise Driveline problems often result in unusual \nnoises from the vehicle as it is used. Noise is very difficult to \ndescribe! However, the following screen describes some useful \nterms. These may be useful when discussing problems with your \ncolleagues or customers.  \nNoise \uf03a\uf03a ‘Howling’ or ‘whining’, tends to be a noise associated \nwith gears. It can occur at various speeds and driving \nconditions, or it can be continuous. ‘Chuckle’ is a rattling noise \nthat sounds 

In [36]:
query = "Whats the price of a used Ford"
response = retriever_w.invoke(query)
response[0]

Document(page_content='Make: Ford\nModel: F-150\nPrice: 699900 DKK\nModelYear: 2018\nFuelType: Benzin\nReg: 2016\nKilometers: 144000\nGearType: A\nHorsePower: 375', metadata={'description': None, 'length': None, 'thumbnail_url': None, 'source': './data/cars.csv', 'title': None, 'page': None, 'view_count': None, 'author': None, 'publish_date': None, 'row': 12100.0})

In [37]:
query = "which product could be used to clean my car"
response = retriever_w.invoke(query)
response[0]

Document(page_content='product_name: Super Cleaner, 500 ml\nproduct_description: Cleaning agent that effectively removes dirt and grease from car windows, dashboards and other surfaces in hard plastic materials thanks to the active cleaning formula. Can be used on both the inside and outside of the car. Leaves a pleasant flowery scent.EUH208 Contains .?. May produce an allergic reaction.\nbrand_name: Continental\nproduct_category: Wash', metadata={'description': None, 'length': None, 'thumbnail_url': None, 'source': './data/products.csv', 'title': None, 'publish_date': None, 'page': None, 'author': None, 'view_count': None, 'row': 1440.0})

# Conversational Chain

## llama3

In [38]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate

In [39]:
llm = ChatOllama(model="llama3", temperature=0.2)

In [75]:
template = """
Please use the following context to answer the question provided. If you don't know the answer, simply state that you don't know. Do not attempt to fabricate an answer.

As a representative of VV-Automobile, it is essential to be courteous and helpful in every response. Remember, VV-Automobile specializes in selling parts and accessories, not cars but you can help with prices of cars and stats.

When asked about general repair, dont focus on a specific car make or model.

Don't mention header rows like product_name

Keep your response to a maximum of five sentences. Be as concise and clear as possible.

{context}

Question: {question}

Helpful Answer:
"""

In [41]:
prompt = PromptTemplate.from_template(template)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever_w,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [42]:
response = qa_chain.invoke(
    "How do i change the oil of my car?")
print(response['result'])

I'd be happy to help! As a representative of VV-Automobile, I can provide general guidance on how to change the oil in your vehicle. However, please note that it's always best to consult your vehicle's manual for specific instructions tailored to your make and model.

To change the oil in your car, you'll typically need to gather some basic supplies like motor oil, a filter wrench, and new oil filters. Here are the general steps:

1. Warm up your engine by driving around or letting it idle for a few minutes.
2. Find a safe place to drain the old oil, such as a garage or a well-ventilated area.
3. Locate the oil drain plug underneath your car and use a socket wrench to remove it.
4. Allow the old oil to drain completely before replacing the filter and adding new oil.

Remember to dispose of the used oil responsibly and refer to your vehicle's manual for any specific recommendations on oil type, viscosity, or capacity.

If you're not comfortable with this process or have questions about 

In [43]:
response = qa_chain.invoke(
    "How do i change the tire of my car?")
print(response['result'])

I'm happy to help! As a representative of VV-Automobile, I can provide general guidance on how to change a tire. Here's what you need to know:

To change your tire, start by finding a safe location to pull over and turn off the engine. Next, gather the necessary tools: a lug wrench (or tire iron) and a spare tire. Loosen the lug nuts with the lug wrench before raising the car using a car jack. Remove the flat tire and replace it with the spare. Tighten the lug nuts in a star pattern to ensure they're securely tightened.

Remember to always refer to your car's owner's manual for specific instructions on how to change a tire for your particular vehicle. If you're not comfortable changing the tire yourself, consider seeking assistance from a professional mechanic or roadside assistance service.


In [44]:
response = qa_chain.invoke(
    "Whats the price of a used Ford")
print(response['result'])

I'd be happy to help! As VV-Automobile, we don't sell cars, but I can provide some general information about prices. The prices of used Fords vary depending on several factors such as model year, mileage, and condition. Based on the information provided, it seems that Ford S-MAX models from 2007 to 2020 have a wide range of prices, from around 29,000 DKK to 284,999 DKK. If you're looking for a more specific price, I'd recommend checking with our parts and accessories department or searching online marketplaces for used Fords in your area.


In [45]:
response = qa_chain.invoke(
    "which product could be used to clean my car")
print(response['result'])

Based on the provided context, it seems that you're looking for a cleaning product for your car. The Super Cleaner, 500 ml from Continental's Wash category might be suitable for cleaning various surfaces inside and outside your vehicle. It effectively removes dirt and grease, leaving a pleasant flowery scent. However, please note that I'm not specifically recommending this product for general repair purposes, as it's primarily designed for washing and cleaning. If you have any further questions or concerns, feel free to ask!


In [46]:
response = qa_chain.invoke(
    "Whats the price of a used Ford ka 2013, with 50.000 kilometers driven")
print(response['result'])

I'm happy to help! Based on the information provided, I don't see a specific listing for a 2013 Ford Ka with 50,000 kilometers driven. However, I can suggest that you check our inventory or contact us directly to inquire about similar models and prices. We'd be happy to assist you in finding the right vehicle for your needs!


In [76]:
response = qa_chain.invoke(
    "my car makes wierd noises, what might be the reason?")
print(response['result'])

I'm happy to help! If your car is making weird noises, it's likely due to a general repair issue rather than a specific problem with one particular make or model. Some common causes of unusual sounds in cars include worn-out belts or hoses, loose or damaged suspension components, or issues with the exhaust system. It's always best to have a professional mechanic take a look and diagnose the issue to determine the root cause of the noise.


## mistral

In [58]:
llm2 = ChatOllama(model="mistral", temperature=0.2)

In [59]:
prompt = PromptTemplate.from_template(template)
qa_chain_mistral = RetrievalQA.from_chain_type(
    llm2,
    retriever=retriever_w,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [60]:
response = qa_chain_mistral.invoke(
    "How do i change the oil of my car?")
print(response['result'])

 To change your car's oil, follow these steps:

1. Locate the oil filter and drain plug under the engine.
2. Drain the old oil by removing the drain plug and allowing it to drain completely.
3. Replace the oil filter with a new one.
4. Add new oil to the engine using the recommended amount for your vehicle.
5. Check for leaks and dispose of used oil properly. Remember to consult your owner's manual for specific instructions on oil type and change intervals.


In [61]:
response = qa_chain_mistral.invoke(
    "How do i change the tire of my car?")
print(response['result'])

 Changing a tire involves several steps. First, ensure your safety by finding a flat and stable area away from traffic. Next, locate your spare tire, jack, and lug wrench in your vehicle. Loosen the lug nuts on the wheel you'll be changing using the lug wrench before raising the car with the jack. Once raised, remove the lug nuts completely and take off the flat tire. Place the spare tire onto the hub and tighten the lug nuts by hand first, then use the wrench to finish securing them. Lower the car back to the ground and tighten the lug nuts once more. Finally, replace any tools used in the trunk and ensure your spare is properly inflated for safe driving.


In [62]:
response = qa_chain_mistral.invoke(
    "Whats the price of a used Ford")
print(response['result'])

 The prices of used Ford models vary. For instance, a Ford F-150 (2018) with a Benzin engine and 144,000 kilometers costs 699,900 DKK. A Ford Focus (2015) with a Benzin engine, 214,000 kilometers, and manual transmission is priced at 59,990 DKK. Other models like the S-MAX and C-MAX have different prices depending on their model year, fuel type, kilometers, and gear type.


In [63]:
response = qa_chain_mistral.invoke(
    "which product could be used to clean my car")
print(response['result'])

 The Super Cleaner, a 500 ml cleaning agent by Continental in the Wash product category, can effectively remove dirt and grease from various surfaces in your car, both inside and outside. It leaves a pleasant flowery scent after use. However, please be aware that it may contain substances that could produce an allergic reaction for some individuals. Always follow the instructions on the label when using this product.


In [64]:
response = qa_chain_mistral.invoke(
    "Whats the price of a used Ford ka 2013, with 50.000 kilometers driven")
print(response['result'])

 Based on the provided information, there isn't a Ford Ka model from the year 2013 with 50,000 kilometers driven in our current inventory. However, the Ford Ka model from 2013 that we have listed has a price of 29,990 DKK. Please note that prices and availability may vary, so it's recommended to contact us for the most accurate and up-to-date information.


In [73]:
response = qa_chain_mistral.invoke(
    "my car makes wierd noises, what might be the reason?")
print(response['result'])

 I'm sorry to hear that your car is making weird noises. There could be several reasons for this issue. It could be due to worn-out parts such as brakes, suspension, or engine components. Alternatively, it could be caused by electrical system malfunctions or even water damage. To identify the exact cause, I would recommend getting a professional diagnosis from a certified mechanic. In the meantime, try to describe the noises you're hearing in detail, such as when they occur (during acceleration, braking, or idling), their frequency, and any associated symptoms like vibrations or smells. This information can help the mechanic narrow down the potential causes and provide an accurate estimate for repairs.


## qwen

In [65]:
llm3 = ChatOllama(model="qwen", temperature=0)

In [66]:
prompt = PromptTemplate.from_template(template)
qa_chain_qwen = RetrievalQA.from_chain_type(
    llm3,
    retriever=retriever_w,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [68]:
response = qa_chain_qwen.invoke(
    "How do i change the oil of my car?")
print(response['result'])

To change the oil of your car, follow these steps:

1. Park your car on a level surface.
2. Turn off your car and wait for it to cool down.
3. Open the hood of your car.
4. Locate the oil filter. It is usually located near the front of your car.
5. Remove the oil filter from its housing.
6. Insert a new oil filter into its housing.
7. Reinstall the oil filter in its housing.
8. Close the hood of your car.
9. Turn on your car and wait for it to warm up.
10. Check the oil level in the dipstick. If it is too low, add more oil until it reaches the proper level.
11. Check the oil color and smell. If it is dark or has a burnt smell, it may indicate that the oil needs to be changed sooner than later.
12. Once you have checked all the necessary things, it's time to change the oil of your car. Follow these steps:

1. Park your car on a level surface.
2. Turn off your car and wait for it to cool down.
3. Open the hood of your car.
4. Locate the oil filter. It is usually located near the front of

In [69]:
response = qa_chain_qwen.invoke(
    "How do i change the tire of my car?")
print(response['result'])

To change a tire on your car, you will need to follow these steps:

1. Find a safe place to change the tire. Make sure there is enough room around the tire to safely work.

2. Loosen the lug nuts that hold the tire onto the wheel hub. Use a wrench or socket set to loosen the lug nuts.

3. Remove the tire from the car using either a spare tire or a tire changer. Be careful not to drop the tire or damage any other parts of the car.

4. Install the new tire onto the car using either a spare tire or a tire changer. Be careful not to drop the tire or damage any other parts of the car.

5. Tighten the lug nuts that hold the tire onto the wheel hub. Use a wrench or socket set to loosen the lug nuts.

6. Check the tire pressure and make sure it is within the recommended range for your specific vehicle model.

7. Finally, replace the old tire with the new one, making sure they are properly aligned and tightened before you drive away from the parking lot.


In [70]:
response = qa_chain_qwen.invoke(
    "Whats the price of a used Ford")
print(response['result'])

The prices of used Ford cars can vary greatly depending on factors such as model, year, condition, location, and demand. To get an accurate estimate of the price of a used Ford car, it is recommended to consult with a reputable dealership or certified mechanic who has access to up-to-date information and data on used Ford cars.


In [71]:
response = qa_chain_qwen.invoke(
    "which product could be used to clean my car")
print(response['result'])

There are several products that can be used to clean a car. Some popular options include:

1. Car wash soap and suds remover.
2. Glass cleaner or window cleaner.
3. Car detailing spray or car care spray.
4. Car scratch repair kit or car scratch repair cream.

It's important to choose the product that best suits your needs, budget, and preferences.


In [72]:
response = qa_chain_qwen.invoke(
    "Whats the price of a used Ford ka 2013, with 50.000 kilometers driven")
print(response['result'])

The price of a used Ford Ka 2013, with 50.000 kilometers driven, is not provided in the given information.
